#Importando as bibliotecas neceessárias:

In [39]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
# importar o dataset para um dataframe
df = pd.read_csv('sao-paulo-properties-april-2019.csv')

# ver as 5 primeiras entradas
df.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


#Análise Exploratória Inicial:

In [29]:
df.isnull().sum()

Price               0
Condo               0
Size                0
Rooms               0
Toilets             0
Suites              0
Parking             0
Elevator            0
Furnished           0
Swimming Pool       0
New                 0
District            0
Negotiation Type    0
Property Type       0
Latitude            0
Longitude           0
dtype: int64

In [30]:
df.dtypes

Price                 int64
Condo                 int64
Size                  int64
Rooms                 int64
Toilets               int64
Suites                int64
Parking               int64
Elevator              int64
Furnished             int64
Swimming Pool         int64
New                   int64
District             object
Negotiation Type     object
Property Type        object
Latitude            float64
Longitude           float64
dtype: object

In [31]:
df.describe().round(2)

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,Latitude,Longitude
count,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00,13640.00
mean,287737.78,689.88,84.37,2.31,2.07,0.98,1.39,0.35,0.15,0.51,0.02,-22.08,-43.60
std,590821.42,757.65,58.44,0.78,0.96,0.83,0.83,0.48,0.35,0.50,0.12,5.87,11.49
min,480.00,0.00,30.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,-46.75,-58.36
25%,1858.75,290.00,50.00,2.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,-23.59,-46.68
50%,8100.00,500.00,65.00,2.00,2.00,1.00,1.00,0.00,0.00,1.00,0.00,-23.55,-46.64
75%,360000.00,835.00,94.00,3.00,2.00,1.00,2.00,1.00,0.00,1.00,0.00,-23.52,-46.56
max,10000000.00,9500.00,880.00,10.00,8.00,6.00,9.00,1.00,1.00,1.00,1.00,0.00,0.00


#Retirando Outliers:

In [40]:
def tirar_outlier(df, lista):
  df_clean = df[lista]
  for col, dados in df_clean.iteritems():
    df_clean = df_clean[((df_clean[col] - df_clean[col].mean()) / df_clean[col].std()).abs() < 3]
    df_clean = df_clean[np.abs(df_clean[col]-df_clean[col].mean()) <= (3*df_clean[col].std())]

  valores_retirados = df.shape[0] - df_clean.shape[0]
  print('Foi retirado {} valores de {} ({:.2f})%'.format(valores_retirados, df.shape[0], valores_retirados/df.shape[0]*100))
  print('Ficamos com {} resultados'.format(df_clean.shape[0]))

  for i in range(0, len(df)):
    if i not in df_clean.index:
        df = df[df.index != i]

  df = df.reset_index(drop=True)
  return df

lista = ['Latitude', 'Longitude']
df_clean = tirar_outlier(df, lista)

Foi retirado 992 valores de 13640 (7.27)%
Ficamos com 12648 resultados


#Correção de Dados:

In [0]:
#

#Primeiros Modelos:

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
X = df_clean.drop(['Price', 'District', 'Negotiation Type', 'Property Type'], axis = 1)
y = df_clean[['Price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)
print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))

R² = 0.219


In [34]:
X.shape

(12648, 12)

In [41]:
import random

R = 0
y = df_clean[['Price']]

while R < 0.95:
  n = random.randint(1, 12)
  X = df_clean.drop(['Price', 'Negotiation Type', 'Property Type'], axis = 1)
  X = X.sample(n = n, axis = 1)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
  modelo = XGBRegressor(random_state=42, silent=True)
  modelo.fit(X_train, y_train)
  y_pred = modelo.predict(X_test)
  R = r2_score(y_test, y_pred)
  if R > 0.5:
    print('R² = {:.3f}'.format(r2_score(y_test, y_pred)))
    lista = list(X.columns)
    lista.sort()
    print(lista)
    print()

ValueError: ignored